<h1> K-Nearest-Neighbors

In [120]:
import numpy as np
import pandas as pd

manuale  = pd.read_csv('manuale.csv', sep=';')
manuale


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15,98,86,high risk
1,35,140,90,13,98,70,high risk
2,29,90,70,8,100,80,high risk
3,30,140,85,7,98,70,high risk
4,35,120,60,61,98,76,low risk
5,23,140,80,701,98,70,high risk
6,23,130,70,701,98,78,mid risk
7,35,85,60,11,102,86,high risk
8,32,120,90,69,98,70,mid risk
9,42,130,80,18,98,70,high risk


In [121]:
#701???

La formula della distanza euclidea tra due punti in uno spazio n-dimensionale è:

$$
d(p_1, p_2) = \sqrt{\sum_{i=1}^{n} (p_{1,i} - p_{2,i})^2}
$$

Per uno spazio bidimensionale, la formula diventa:

$$
d(p_1, p_2) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$

In [122]:
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):  # Escludere l'ultima colonna (la classe target)
        distance += (row1[i] - row2[i]) ** 2
    return np.sqrt(distance)

La funzione knn_predict implementa l’algoritmo KNN. Dato un set di punti di training  X , le rispettive etichette  y , un set di punti di test e  k , restituisce le predizioni.

In [123]:
from collections import Counter

def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        # Calcola la distanza tra il punto di test e tutti i punti di training
        distances = [(euclidean_distance(test_point, x_train), y_train[i]) for i, x_train in enumerate(X_train)]
        # Ordina le distanze in ordine crescente
        distances.sort(key=lambda x: x[0])
        # Seleziona i primi k vicini
        k_nearest = [dist[1] for dist in distances[:k]]
        # Predici la classe in base alla votazione a maggioranza
        most_common = Counter(k_nearest).most_common(1)[0][0]
        predictions.append(most_common)
    return predictions

Cross-validation con k-fold divide il dataset in  k -fold, usa ciascun fold come test set una volta, mentre il resto dei fold viene usato come training set. Calcola l’accuratezza media su tutti i fold.

In [124]:
# Funzione per suddividere i dati in k-fold
def k_fold_split(X, y, k):
    fold_size = len(X) // k
    X_folds = []
    y_folds = []
    for i in range(k):
        X_folds.append(X[i * fold_size: (i + 1) * fold_size])
        y_folds.append(y[i * fold_size: (i + 1) * fold_size])
    return X_folds, y_folds

def cross_validate_knn(X, y, k_neighbors, num_folds=5):
    X_folds, y_folds = k_fold_split(X, y, num_folds)
    
    accuracies = []  # Per memorizzare le accuratezze per ciascun fold
    for i in range(num_folds):
        # Usa il fold i-esimo come test set
        X_test_fold = X_folds[i]
        y_test_fold = y_folds[i]
        
        # Usa tutti gli altri fold come training set
        X_train_folds = np.concatenate([X_folds[j] for j in range(num_folds) if j != i])
        y_train_folds = np.concatenate([y_folds[j] for j in range(num_folds) if j != i])
        
        # Prevedi con KNN
        y_pred = knn_predict(X_train_folds, y_train_folds, X_test_fold, k_neighbors)
        
        # Calcola l'accuratezza per questo fold
        accuracy = np.mean(np.array(y_pred) == np.array(y_test_fold))
        accuracies.append(accuracy)
    
    # Restituisci l'accuratezza media sui fold
    return np.mean(accuracies)


Questa funzione trova il miglior  k  testando diversi valori di  k  e valutando le prestazioni medie tramite cross-validation

In [125]:
def find_best_k(X, y, k_values, num_folds=5):
    best_k = k_values[0]
    best_accuracy = 0
    
    for k in k_values:
        # Esegui la cross-validation per ciascun k
        accuracy = cross_validate_knn(X, y, k, num_folds)
        print(f'Accuracy for k={k}: {accuracy:.4f}')
        
        # Se l'accuratezza per questo k è migliore, aggiorna il miglior k
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k
    
    print(f'Miglior valore di k: {best_k} con un\'accuratezza di {best_accuracy:.4f}')
    return best_k

In [126]:

# Separa le feature (X) e le etichette (y)
X = manuale.iloc[:, :-1].values  # Tutte le colonne tranne l'ultima (feature)
y = manuale.iloc[:, -1].values    # L'ultima colonna (target)
# Esegui la ricerca del miglior valore di k
k_values = list(range(1, 11))  # Prova k da 1 a 10
best_k = find_best_k(X, y, k_values, num_folds=5)  # Cross-validation a 5 fold

# **Valutazione finale**: Usa il miglior k trovato per fare predizioni su tutto il dataset
y_final_pred = knn_predict(X, y, X, best_k)

# Calcola e stampa l'accuratezza finale usando lo stesso dataset
final_accuracy = np.mean(y_final_pred == y)
print(f'Accuratezza finale sui dati (con k={best_k}): {final_accuracy:.4f}')


Accuracy for k=1: 0.5667
Accuracy for k=2: 0.5667
Accuracy for k=3: 0.5667
Accuracy for k=4: 0.5667
Accuracy for k=5: 0.6333
Accuracy for k=6: 0.6000
Accuracy for k=7: 0.6333
Accuracy for k=8: 0.6000
Accuracy for k=9: 0.5667
Accuracy for k=10: 0.5333
Miglior valore di k: 5 con un'accuratezza di 0.6333
Accuratezza finale sui dati (con k=5): 0.8333


In [ ]:
#proviamo a normalaizzare i dati:

# Normalizzazione manuale
def min_max_normalize(data):
    # Calcola il minimo e il massimo per ogni colonna
    data_min = np.min(data, axis=0)
    data_max = np.max(data, axis=0)
    
    # Applica la formula della normalizzazione
    normalized_data = (data - data_min) / (data_max - data_min)
    return normalized_data


# Separa le feature (X) e le etichette (y)
X = manuale.iloc[:, :-1].values  # Tutte le colonne tranne l'ultima (feature)
y = manuale.iloc[:, -1].values    # L'ultima colonna (target)

normalized_manuale = min_max_normalize(X)
print("Dati Normalizzati:\n", normalized_manuale)

#applichiamo knn sul dataset normalizzato:


# Esegui la ricerca del miglior valore di k
k_values = list(range(1, 11))  # Prova k da 1 a 10
best_k = find_best_k(X, y, k_values, num_folds=5)  # Cross-validation a 5 fold

# **Valutazione finale**: Usa il miglior k trovato per fare predizioni su tutto il dataset
y_final_pred = knn_predict(normalized_manuale, y, normalized_manuale, best_k)

# Calcola e stampa l'accuratezza finale usando lo stesso dataset
final_accuracy = np.mean(y_final_pred == y)
print(f'Accuratezza finale sui dati (con k={best_k}): {final_accuracy:.4f}')



Dati Normalizzati:
 [[0.375      0.85714286 0.6        0.01152738 0.         0.83333333]
 [0.625      1.         0.8        0.00864553 0.         0.16666667]
 [0.475      0.28571429 0.4        0.00144092 0.5        0.58333333]
 [0.5        1.         0.7        0.         0.         0.16666667]
 [0.625      0.71428571 0.2        0.0778098  0.         0.41666667]
 [0.325      1.         0.6        1.         0.         0.16666667]
 [0.325      0.85714286 0.4        1.         0.         0.5       ]
 [0.625      0.21428571 0.2        0.00576369 1.         0.83333333]
 [0.55       0.71428571 0.8        0.08933718 0.         0.16666667]
 [0.8        0.85714286 0.6        0.01585014 0.         0.16666667]
 [0.325      0.28571429 0.2        1.         0.         0.41666667]
 [0.225      0.71428571 0.6        0.         0.         0.16666667]
 [0.375      0.57142857 0.78       1.         0.         0.45833333]
 [0.25       0.71428571 0.5        1.         0.5        0.16666667]
 [0.95       0

<h1> Gaussian Naive Bayes

In [128]:
def calcola_parametri(X, y):
    # Identifica le classi uniche
    classes = np.unique(y)
    
    # Dictionario per contenere i parametri per ogni classe
    parameters = {}
    
    # Calcola media, varianza e probabilità a priori per ogni classe
    for cls in classes:
        X_c = X[y == cls]  # Dati appartenenti alla classe corrente
        parameters[cls] = {
            'mean': X_c.mean(axis=0),  # Media di ciascuna feature
            'var': X_c.var(axis=0),    # Varianza di ciascuna feature
            'prior': X_c.shape[0] / X.shape[0]  # Probabilità a priori
        }
    
    return parameters

In [129]:
def gaussian_probability(x, mean, var):
    # Aumenta epsilon per evitare varianze troppo piccole che causano problemi numerici
    epsilon = 1e-1 # Valore piccolo ma più grande di quello precedente
    coefficient = 1 / np.sqrt(2 * np.pi * (var + epsilon))
    exponent = np.exp(-((x - mean) ** 2) / (2 * (var + epsilon)))
    return coefficient * exponent

In [130]:
def class_probability(x, parameters):
    probabilities = {}
    
    # Calcola la probabilità per ciascuna classe
    for cls, params in parameters.items():
        prior = np.log(params['prior'])  # Usa log per la probabilità a priori
        likelihood = np.sum(np.log(gaussian_probability(x, params['mean'], params['var'])))
        probabilities[cls] = prior + likelihood
    
    return probabilities


In [131]:
def predict(X, parameters):
    predictions = []
    # Per ogni esempio nei dati di input
    for x in X:
        class_probabilities = class_probability(x, parameters)
        # Seleziona la classe con la probabilità a posteriori massima
        predictions.append(max(class_probabilities, key=class_probabilities.get))
    
    return np.array(predictions)


In [132]:
# Calcola i parametri (media, varianza, probabilità a priori) per ogni classe
parameters = calcola_parametri(X, y)

# Prevedi le classi per i nuovi esempi o usa i dati esistenti per il test
X_test = X  # Puoi usare i tuoi dati esistenti o nuovi esempi

# Predici le classi
predictions = predict(X_test, parameters)

# Visualizza le predizioni
print("Predizioni:", predictions)

# Confronta con le etichette vere (opzionale)
print("Etichette vere:", y)


Predizioni: ['high risk' 'high risk' 'high risk' 'high risk' 'low risk' 'low risk'
 'low risk' 'high risk' 'low risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'mid risk' 'high risk' 'low risk' 'high risk' 'high risk'
 'mid risk' 'low risk' 'high risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'low risk' 'low risk' 'low risk' 'mid risk' 'low risk']
Etichette vere: ['high risk' 'high risk' 'high risk' 'high risk' 'low risk' 'high risk'
 'mid risk' 'high risk' 'mid risk' 'high risk' 'low risk' 'mid risk'
 'low risk' 'mid risk' 'mid risk' 'low risk' 'high risk' 'high risk'
 'mid risk' 'low risk' 'high risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'low risk' 'low risk' 'low risk' 'low risk' 'low risk']


In [133]:
def calculate_accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)  # Conta le predizioni corrette
    accuracy = correct_predictions / len(y_true)  # Divide per il numero totale di esempi
    return accuracy

In [134]:
# Supponiamo che tu abbia le predizioni già pronte
predictions = predict(X, parameters)  # Prevedi con il modello Gaussian Naive Bayes

# Valuta l'accuracy
accuracy = calculate_accuracy(y, predictions)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.77


Matrice di confusione

In [135]:
def confusion_matrix(y_true, y_pred):
    # Identifica le classi uniche
    classes = np.unique(np.concatenate((y_true, y_pred)))
    # Crea una matrice di confusione vuota
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    
    # Popola la matrice
    for true, pred in zip(y_true, y_pred):
        true_index = np.where(classes == true)[0][0]  # Trova l'indice della classe vera
        pred_index = np.where(classes == pred)[0][0]  # Trova l'indice della classe prevista
        matrix[true_index, pred_index] += 1
    
    return matrix, classes

# Usa la funzione per calcolare la matrice di confusione
cm, classes = confusion_matrix(y, predictions)

# Stampa la matrice di confusione
print("Matrice di confusione:")
print(cm)

# Opzionale: stampa le etichette delle classi
print("Etichette delle classi:", classes)


Matrice di confusione:
[[ 8  1  1]
 [ 0 12  1]
 [ 1  3  3]]
Etichette delle classi: ['high risk' 'low risk' 'mid risk']
